In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import altair as alt
alt.data_transformers.enable('json')
alt.renderers.enable('jupyterlab')
from sublimpy import tidy, utils
import pytz
import datetime as dt

seconds_in_timestep = 60*30
from metpy.constants import density_water

from metpy.units import units
import pint_pandas
from metpy import constants
from scipy import interpolate
np.set_printoptions(suppress=True,precision=10)

# Parameters for calculations

In [ ]:
HEIGHTS = [1,3,10]
HORIZ_GRID_SPACING = 50
VERT_GRID_SPACING = 20
start_date = '20221130'
end_date = '20230509'

data_cutoff_date = '20230508'
    # streamwise-coords
# tidy_df = pd.read_parquet(f"tidy_df_{start_date}_{end_date}_planar_fit_multiplane.parquet")
    # slope-adjusted earthwise-coords
tidy_df = pd.read_parquet(f"tidy_df_{start_date}_{end_date}_planar_fit.parquet")
    # sonic coords
# tidy_df = pd.read_parquet(f"tidy_df_{start_date}_{end_date}_noplanar_fit.parquet")

# method_numerical_advection = 'wind_divergence'      # q * d/dx_i (u_i)
# method_numerical_advection = 'scalar_divergence'    # u_i * d/dx_i (q)
# method_numerical_advection = 'summed'               # q * d/dx_i (u_i) + u_i * d/dx_i (q)
# method_numerical_advection = 'derivative'           # d/dx_i (q * u_i)

# Prepare data

## Open SOS Measurement Dataset

In [ ]:
# convert time column to datetime
tidy_df['time'] = pd.to_datetime(tidy_df['time'])
tidy_df = utils.modify_df_timezone(tidy_df, pytz.UTC, 'US/Mountain')
# limit data to our dates of interest, based on continuous snow cover at Kettle Ponds
tidy_df = tidy_df.set_index('time').sort_index().loc[start_date:data_cutoff_date].reset_index()

## Add combined blowing snow flux variable
tidy_df = tidy.tidy_df_add_variable(
    tidy_df,
    (
        tidy_df.query("variable == 'SF_avg_1m_ue'")['value'].values + 
        tidy_df.query("variable == 'SF_avg_2m_ue'")['value'].values
    ), 
    'SF_avg_ue', 'snow flux', 1, 'ue',
)

In [ ]:
tidy_df

## Add absolute humidity measurements by converting hygrometer measurements

In [ ]:
tower_height_keys = tidy_df[tidy_df.measurement=='specific humidity'].groupby(['tower', 'height']).indices.keys()
for t, h in tower_height_keys:
    this_tower_height_tidy_df = tidy_df.query(
            f"tower == '{t}'"
        ).query(
            f"height == {h}"
        )
    specific_humidity_values = this_tower_height_tidy_df.query(
            "measurement == 'specific humidity'"
        ).set_index('time')[['value']].rename(columns={'value': 'specific humidity'})
    air_density_values = this_tower_height_tidy_df.query(
            "measurement == 'air density'"
        ).set_index('time')[['value']].rename(columns={'value': 'air density'})
    combined_df = specific_humidity_values.join(air_density_values)

    abs_humidity_values = (
        combined_df['specific humidity'].values * units('g/g')
    ).to('g/kg') * (
        combined_df['air density'].values * units('kg/m^3')
    ).m

    tidy_df = tidy.tidy_df_add_variable(
        tidy_df,
        abs_humidity_values,
        f"absolutehumidity_{int(h)}m_{t}",
        'absolute humidity',
        int(h),
        t
    )

## Calibrate gas analyzer measurements

We calibrate by assuming that all gas analyzers have the same seasonal mean as the corresponding hygrometer measurement on the central tower (at a given height)

In [ ]:
hygrometer_absolute_humidity_mean = (
    1000 * tidy_df[tidy_df.measurement=='specific humidity'].groupby(['tower', 'height'])[['value']].mean() *\
    tidy_df[tidy_df.measurement=='air density'].groupby(['tower', 'height'])[['value']].mean()
).reset_index().query("tower == 'c'")

In [ ]:
ec_absolute_humidity_mean = tidy_df[tidy_df.measurement=='Water vapor density'].groupby(['variable', 'tower', 'height'])[['value']].mean().reset_index()

In [ ]:
corrections_df = ec_absolute_humidity_mean.merge(
    hygrometer_absolute_humidity_mean[['height', 'value']].rename(columns={'value': 'truth'}),
    on='height'
)
corrections_df['offset'] = corrections_df['value'] - corrections_df['truth']
corrections_df

Update dataset with corrections

In [ ]:
src = tidy_df[tidy_df.measurement=='Water vapor density']
src = src[src.height.isin([1,3,10])]
src

In [ ]:
alt.Chart(
    (
    1000 * tidy_df[tidy_df.measurement=='specific humidity'].groupby(['tower', 'height'])[['value']].mean() *\
    tidy_df[tidy_df.measurement=='air density'].groupby(['tower', 'height'])[['value']].mean()
    ).reset_index()
).mark_point(shape='square', filled=True, color='black', size=20).encode(
    alt.X("value:Q"),
    alt.Y("height:Q")
).properties(width=150, height = 150)\
+ alt.Chart(
    tidy_df[tidy_df.measurement=='Water vapor density'].groupby(['variable', 'tower', 'height'])[['value']].mean().reset_index()
).mark_circle(size=40).encode(
    alt.X("value:Q"),
    alt.Y("height:Q"),
    alt.Color('tower:N')
).properties(width=150, height = 150)

In [ ]:


alt.Chart(
    src[ src.time > '20221212' ][ src.time < '20221214' ]
).mark_line().encode(
    alt.X("time:T"),
    alt.Y("value:Q"),
    alt.Color("height:N"),
    detail='variable'
)

In [ ]:
for idx, row in corrections_df.iterrows():
    src = tidy_df.query(f"variable == '{row['variable']}'")
    src = src.assign(value = src.value - row['offset'])
    tidy_df = tidy_df[tidy_df.variable != row['variable']]
    tidy_df = pd.concat([tidy_df, src])

In [ ]:
alt.Chart(
    (
    1000 * tidy_df[tidy_df.measurement=='specific humidity'].groupby(['tower', 'height'])[['value']].mean() *\
    tidy_df[tidy_df.measurement=='air density'].groupby(['tower', 'height'])[['value']].mean()
    ).reset_index()
).mark_point(shape='square', filled=True, color='black', size=20).encode(
    alt.X("value:Q"),
    alt.Y("height:Q")
).properties(width=150, height = 150)\
+ alt.Chart(
    tidy_df[tidy_df.measurement=='Water vapor density'].groupby(['variable', 'tower', 'height'])[['value']].mean().reset_index()
).mark_circle(size=40).encode(
    alt.X("value:Q"),
    alt.Y("height:Q"),
    alt.Color('tower:N')
).properties(width=150, height = 150)

In [ ]:
src = tidy_df[tidy_df.measurement=='Water vapor density']
src = src[src.height.isin([1,3,10])]
abs_hum = alt.Chart(
    src[ src.time > '20221212' ][ src.time < '20221214' ]
).mark_line(strokeWidth=0.5).encode(
    alt.X("time:T"),
    alt.Y("value:Q").title("Absolute humidity (g/m^3)").scale(zero=False),
    alt.Color("height:N"),
    alt.Shape('tower:N'),
    detail='variable'
).properties(width=600)

src = tidy_df[tidy_df.measurement=='snow depth']
snowdepth = alt.Chart(
    src[ src.time > '20221212' ][ src.time < '20221214' ]
).mark_line(strokeWidth=0.5).encode(
    alt.X("time:T"),
    alt.Y("value:Q").title("Snow depth (m)"),
    alt.Shape('tower:N'),
    detail='variable'
).properties(width=600, height=150)

(snowdepth & abs_hum).resolve_scale(color='independent', shape='independent')

# Identify categories for timestamps

In [ ]:
# Identify lists of timestamps for different categories
bs_times = tidy_df.query("variable == 'SF_avg_ue'").query("value > 0").time
nobs_times = tidy_df.query("variable == 'SF_avg_ue'").query("value == 0").time

decoupled_times = tidy_df.query("variable == 'omega_3m_c'").query("value < 0.43").time
weaklycoupled_times = tidy_df.query("variable == 'omega_3m_c'").query("value >= 0.43").query("value <= 0.61").time
coupled_times = tidy_df.query("variable == 'omega_3m_c'").query("value > 0.61").time

ri_stable_times = tidy_df.query("variable == 'Ri_3m_c'").query("value > 0.25").time
ri_unstable_times = tidy_df.query("variable == 'Ri_3m_c'").query("value < -0.01").time
ri_neutral_times = tidy_df.query("variable == 'Ri_3m_c'").query("value >= -0.01").query("value <= 0.25").time

tgrad_stable_times = tidy_df.query("variable == 'temp_gradient_3m_c'").query("value > 0.01").time
tgrad_unstable_times = tidy_df.query("variable == 'temp_gradient_3m_c'").query("value < -0.01").time
tgrad_neutral_times = tidy_df.query("variable == 'temp_gradient_3m_c'").query("value >= -0.01").query("value <= 0.01").time

december_times = tidy_df[tidy_df.time.dt.month == 12].time
january_times = tidy_df[tidy_df.time.dt.month == 1].time
february_times = tidy_df[tidy_df.time.dt.month == 2].time
march_times = tidy_df[tidy_df.time.dt.month == 3].time
april_times = tidy_df[tidy_df.time.dt.month == 4].time

midwinter_times = tidy_df[tidy_df.time < '20230320'].time
spring_times = tidy_df[tidy_df.time > '20230320'].time

In [ ]:
precip_df = xr.open_dataset("/Users/elischwat/Development/data/sublimationofsnow/precip_danny/precipitation_rate_gts_w23.nc")['corrected_prcp_rate_m2'].to_dataframe()

is_snowing_dates = pd.concat([
    precip_df.query("corrected_prcp_rate_m2 > 0").index.to_series(),
    precip_df.query("corrected_prcp_rate_m2 > 0").index.to_series() + dt.timedelta(minutes=30)
])
    
is_not_snowing_dates = pd.concat([
    precip_df.query("corrected_prcp_rate_m2 <= 0").index.to_series(),
    precip_df.query("corrected_prcp_rate_m2 <= 0").index.to_series() + dt.timedelta(minutes=30)
])

# Create tables

## Instrument location info (georeferenced)
We use a file with theodolite/GPS readings provided by NCAR. 

In [ ]:
instrument_loc_df = pd.read_csv("~/Development/data/sublimationofsnow/SOSm.txt", names = ['ec', 'x', 'y', 'z'])
instrument_loc_df = instrument_loc_df[ 
    instrument_loc_df['ec'].str.startswith('CS')
    |
    instrument_loc_df['ec'].str.startswith('DS') 
    |
    instrument_loc_df['ec'].str.startswith('UWS') 
    |
    instrument_loc_df['ec'].str.startswith('UES') 
]
instrument_loc_df = instrument_loc_df[ 
    instrument_loc_df['ec'].str.endswith('T') 
    |
    instrument_loc_df['ec'].str.endswith('B') 
]
instrument_loc_df['top or bottom'] = instrument_loc_df['ec'].str[-1]
instrument_loc_df['tower'] = instrument_loc_df['ec'].apply(lambda str: str.split('S')[0].lower())
instrument_loc_df['height'] = instrument_loc_df['ec'].apply(lambda str: int(str.split('S')[1][:-1]))
instrument_loc_df = instrument_loc_df.drop(columns='ec')
instrument_loc_df = instrument_loc_df.pivot(index=['height', 'tower'], columns='top or bottom').reset_index()
instrument_loc_df = instrument_loc_df.set_index(['height', 'tower']).groupby(level=0, axis=1).mean()
instrument_loc_df

## Wind field measurements

In [ ]:
wind_field_df = tidy_df[tidy_df.measurement.isin(['u','v','w']) & tidy_df.height.isin(HEIGHTS)]
wind_field_df = round(wind_field_df.pivot_table(index='time', columns=['height', 'tower', 'measurement'], values='value'), 4)
wind_field_df

## Turbulent water vapor flux measurements

In [ ]:
turb_flux_field_df = tidy_df[tidy_df.measurement.isin(['u_h2o_','v_h2o_','w_h2o_']) & tidy_df.height.isin(HEIGHTS)]
turb_flux_field_df = round(turb_flux_field_df.pivot_table(index='time', columns=['height', 'tower', 'measurement'], values='value'), 4)
turb_flux_field_df

## Turbulent temperature flux measurements

In [ ]:
temp_turb_flux_field_df = tidy_df[tidy_df.measurement.isin(['u_tc_','v_tc_','w_tc_']) & tidy_df.height.isin(HEIGHTS)]
temp_turb_flux_field_df = round(temp_turb_flux_field_df.pivot_table(index='time', columns=['height', 'tower', 'measurement'], values='value'), 4)
temp_turb_flux_field_df

## Humidity measurements

In [ ]:
abs_hum_field_df = tidy_df[tidy_df.measurement.isin(['Water vapor density']) & tidy_df.height.isin(HEIGHTS)]
abs_hum_field_df.measurement = 'q'
abs_hum_field_df = round(
    abs_hum_field_df.pivot_table(
        index='time', columns=['height', 'tower', 'measurement'], values='value'
    ), 
    4
)
abs_hum_field_df

## Advective flux measurements

In [ ]:
ls = []
for h in wind_field_df.columns.get_level_values('height').unique():
    for t in wind_field_df.columns.get_level_values('tower').unique():
        this_wind_df = wind_field_df[(h,t)].copy()
        this_abs_hum_df = abs_hum_field_df[(h,t)].copy()  
        this_wind_df['uq'] = this_wind_df['u']*this_abs_hum_df['q']
        this_wind_df['vq'] = this_wind_df['v']*this_abs_hum_df['q']
        this_wind_df['wq'] = this_wind_df['w']*this_abs_hum_df['q']
        new = pd.concat([this_wind_df], axis=1, keys=[(h,t)])
        ls.append(new.drop(columns=[(h,t,'u'),(h,t,'v'),(h,t,'w')]))

adv_flux_field_df = ls[0]
for l in ls[1:]:
    adv_flux_field_df = adv_flux_field_df.join(l)
adv_flux_field_df.columns = adv_flux_field_df.columns.set_names('height', level=0)
adv_flux_field_df.columns = adv_flux_field_df.columns.set_names('tower', level=1)
adv_flux_field_df

## Dry air density measurements

In [ ]:
# gather dry air density measurements
dryair_density_field_df = tidy_df[tidy_df.measurement.isin(['dry air density']) & tidy_df.height.isin(HEIGHTS)]
dryair_density_field_df.measurement = 'rho'
dryair_density_field_df = round(
    dryair_density_field_df.pivot_table(
        index='time', columns=['height', 'tower', 'measurement'], values='value'
    ), 
    4
)

# duplicate the dry air density measurements across the towers (THIS IS NAIVE)
dryair_density_for_tower_d = dryair_density_field_df.copy()
dryair_density_for_tower_d.columns = pd.MultiIndex.from_tuples([(cs[0], 'd', cs[2]) for cs in dryair_density_for_tower_d.columns])

dryair_density_for_tower_uw = dryair_density_field_df.copy()
dryair_density_for_tower_uw.columns = pd.MultiIndex.from_tuples([(cs[0], 'uw', cs[2]) for cs in dryair_density_for_tower_d.columns])

dryair_density_for_tower_ue = dryair_density_field_df.copy()
dryair_density_for_tower_ue.columns = pd.MultiIndex.from_tuples([(cs[0], 'ue', cs[2]) for cs in dryair_density_for_tower_d.columns])

dryair_density_field_df = dryair_density_field_df.join(
    dryair_density_for_tower_d
).join(
    dryair_density_for_tower_ue
).join(
    dryair_density_for_tower_uw
)

dryair_density_field_df.columns = dryair_density_field_df.columns.set_names(['height', 'tower', 'measurement'])
dryair_density_field_df

## Mixing ratio measurements

In [ ]:
mixing_ratio_field_df = abs_hum_field_df.droplevel(2, 1) / dryair_density_field_df.droplevel(2, 1)

mixing_ratio_field_df.columns = pd.MultiIndex.from_product(mixing_ratio_field_df.columns.levels + [['r']])
mixing_ratio_field_df.columns = mixing_ratio_field_df.columns.set_names('measurement', level=2)
mixing_ratio_field_df

# Divergence calculations

In [ ]:
def divergence(f,sp):
    """ 
    From: https://stackoverflow.com/a/67971515
    Computes divergence of vector field 
    f: array -> vector field components [Fx,Fy,Fz,...]
    sp: array -> spacing between points in respecitve directions [spx, spy,spz,...]
    """
    num_dims = len(f)
    return np.ufunc.reduce(np.add, [np.gradient(f[i], sp[i], axis=i) for i in range(num_dims)])

## Calculate interpolated fields

In [ ]:
# Initialize lists to store results
wind_fields = []
turbulent_latentheat_flux_fields = []
turbulent_sensibleheat_flux_fields = []
abshumidity_fields = []
advectedflux_fields = []
dryairdensity_fields = []
mixingratio_fields = []

grid_spacings = []
timestamps = []


field_dataframes ={
    'wind' :            wind_field_df,
    'turb_flux' :       turb_flux_field_df,
    'temp_turb_flux' :  temp_turb_flux_field_df,
    'abs_hum' :         abs_hum_field_df,
    'adv_flux' :        adv_flux_field_df,
    'dryair_density' :  dryair_density_field_df,
    'mixing_ratio' :    mixing_ratio_field_df,
}

for i in wind_field_df.index:
    # sometimes we don't have all measurements - this ensures we only retrieve data that exists
    if all([i in df.index for df in field_dataframes.values()]):
        values_dataframes = {}
        
        # Isolate all measurementsfor this timestamp
        for key in field_dataframes.keys():
            values_dataframes[key] = pd.DataFrame(
                field_dataframes[key].loc[i]
            ).reset_index().set_index(['height', 'tower']).pivot(columns='measurement')
            values_dataframes[key].columns = values_dataframes[key].columns.droplevel(0)

        # Combine all measurements of fields and instrument locations into one dataframe
        points_and_fields = instrument_loc_df.join(
            values_dataframes['wind'], how='right' # join on right df, so we drop instruments that we don't have measurements for
        )
        for key in field_dataframes.keys():
            if key != 'wind':
                points_and_fields = points_and_fields.join(values_dataframes[key])

        # Create a meshgrid for the interpolation
        xx, yy, zz = np.meshgrid(
            np.linspace(points_and_fields.x.min(), points_and_fields.x.max(), HORIZ_GRID_SPACING),
            np.linspace(points_and_fields.y.min(), points_and_fields.y.max(), HORIZ_GRID_SPACING),
            np.linspace(points_and_fields.z.min(), points_and_fields.z.max(), VERT_GRID_SPACING)
        )
        points = np.transpose(np.vstack((points_and_fields.x, points_and_fields.y, points_and_fields.z)))

        # Interpolate wind field
        u_interp = interpolate.griddata(points, points_and_fields['u'], (xx, yy, zz), method='linear')
        v_interp = interpolate.griddata(points, points_and_fields['v'], (xx, yy, zz), method='linear')
        w_interp = interpolate.griddata(points, points_and_fields['w'], (xx, yy, zz), method='linear')

        # Interpolate turbulent latent heat flux fields
        u_q__interp = interpolate.griddata(points, points_and_fields['u_h2o_'], (xx, yy, zz), method='linear')
        v_q__interp = interpolate.griddata(points, points_and_fields['v_h2o_'], (xx, yy, zz), method='linear')
        w_q__interp = interpolate.griddata(points, points_and_fields['w_h2o_'], (xx, yy, zz), method='linear')

        # Interpolate turbulent sensible heat flux fields
        u_tc__interp = interpolate.griddata(points, points_and_fields['u_tc_'], (xx, yy, zz), method='linear')
        v_tc__interp = interpolate.griddata(points, points_and_fields['v_tc_'], (xx, yy, zz), method='linear')
        w_tc__interp = interpolate.griddata(points, points_and_fields['w_tc_'], (xx, yy, zz), method='linear')

        # Interpolate advected flux field
        uq_interp = interpolate.griddata(points, points_and_fields['uq'], (xx, yy, zz), method='linear')
        vq_interp = interpolate.griddata(points, points_and_fields['vq'], (xx, yy, zz), method='linear')
        wq_interp = interpolate.griddata(points, points_and_fields['wq'], (xx, yy, zz), method='linear')

        # Interpolate abs. humidity field
        q_interp = interpolate.griddata(points, points_and_fields['q'], (xx, yy, zz), method='linear')

        # Interpolate dry air density field
        rho_interp = interpolate.griddata(points, points_and_fields['rho'], (xx, yy, zz), method='linear')

        # Interpolate (water vapor) mixing ratio field
        r_interp = interpolate.griddata(points, points_and_fields['r'], (xx, yy, zz), method='linear')

        # Combine interpolated components into vector fields
        F = np.array([u_interp, v_interp, w_interp])
        Fq = np.array([uq_interp, vq_interp, wq_interp])
        F_q_ = np.array([u_q__interp, v_q__interp, w_q__interp])
        F_tc_ = np.array([u_tc__interp, v_tc__interp, w_tc__interp])

        # Record grid spacing        
        sp_x = np.diff(xx[0,:,0]).mean()
        sp_y = np.diff(yy[:,0,0]).mean()
        sp_z = np.diff(zz[0,0,:]).mean()
        sp = [sp_x, sp_y, sp_z]

        # Append interpolated fields to our results lists
        wind_fields.append(F)
        advectedflux_fields.append(Fq)
        turbulent_latentheat_flux_fields.append(F_q_)
        turbulent_sensibleheat_flux_fields.append(F_tc_)
        abshumidity_fields.append(q_interp)
        dryairdensity_fields.append(rho_interp)
        mixingratio_fields.append(r_interp)
        grid_spacings.append(sp)
        timestamps.append(i)

## Calculate divergences


$$
\overline{S} = 
\overline{\rho} \frac{\partial \overline{s}}{\partial t} 
+ \overline{u_i} \space \overline{\rho} \frac{\partial \overline{s}}{\partial x_i} 
- \overline{s} \frac{\partial }{\partial z} \Big( 
    \frac{\overline{\rho}}{\overline{T}} ( 1 + \mu \overline{s}) \overline{w'T'} + \mu \overline{w'q'} 
\Big)
+ \frac{\partial \overline{w'q'}}{\partial z}  
$$

In [ ]:
mu = 1/0.622
T = 273.15

In [ ]:
advective_term_lateral_ls = []
advective_term_vertical_ls = []
advective_term_total_ls = []
airdensityflux_term_vertical_ls = []
turbulent_term_vertical_ls = []

for i in range(0, len(wind_fields)):
    wind_field          = wind_fields[i]
    advectedflux_field  = advectedflux_fields[i]
    turbulent_latentheat_flux_field = turbulent_latentheat_flux_fields[i]
    turbulent_sensibleheat_flux_field = turbulent_sensibleheat_flux_fields[i]
    abshumidity_field   = abshumidity_fields[i]
    dryairdensity_field   = dryairdensity_fields[i]
    mixingratio_field   = mixingratio_fields[i]
    grid_spacing        = grid_spacings[i]

    # create wind fields with just lateral and vertical components
    wind_field_vertical_only = wind_field.copy()
    wind_field_lateral_only  = wind_field.copy()
    wind_field_vertical_only[0] = 0
    wind_field_vertical_only[1] = 0
    wind_field_lateral_only[2] = 0

    advectedflux_field_vertical_only = advectedflux_field.copy()
    advectedflux_field_lateral_only  = advectedflux_field.copy()
    advectedflux_field_vertical_only[0] = 0
    advectedflux_field_vertical_only[1] = 0
    advectedflux_field_lateral_only[2] = 0

    turbulent_latentheat_flux_field_vertical_only = turbulent_latentheat_flux_field.copy()
    turbulent_latentheat_flux_field_lateral_only = turbulent_latentheat_flux_field.copy()
    turbulent_latentheat_flux_field_vertical_only[0] = 0
    turbulent_latentheat_flux_field_vertical_only[1] = 0
    turbulent_latentheat_flux_field_lateral_only[2] = 0

    turbulent_sensibleheat_flux_field_vertical_only = turbulent_sensibleheat_flux_field.copy()
    turbulent_sensibleheat_flux_field_lateral_only = turbulent_sensibleheat_flux_field.copy()
    turbulent_sensibleheat_flux_field_vertical_only[0] = 0
    turbulent_sensibleheat_flux_field_vertical_only[1] = 0
    turbulent_sensibleheat_flux_field_lateral_only[2] = 0

    advective_term_lateral = wind_field*dryairdensity_field*np.gradient(
        mixingratio_field,  grid_spacing[0],    axis=0
    ) + np.gradient(
        mixingratio_field,  grid_spacing[1],    axis=1
    )
    
    advective_term_vertical = wind_field*dryairdensity_field*np.gradient(
        mixingratio_field,  grid_spacing[2],    axis=2
    )

    advective_term_total = advective_term_lateral + advective_term_vertical

    airdensityflux_term_vertical = mixingratio_field * np.gradient(
        (
            (dryairdensity_field/T) * (1 + mu*mixingratio_field) * turbulent_sensibleheat_flux_field + mu*turbulent_latentheat_flux_field
        ),
        grid_spacing[2],
        axis=2
    )

    turbulent_term_vertical = np.gradient(
        turbulent_latentheat_flux_field,
        grid_spacing[2],
        axis=2
    )

    advective_term_lateral_ls.append(advective_term_lateral)
    advective_term_vertical_ls.append(advective_term_vertical)
    advective_term_total_ls.append(advective_term_total)
    airdensityflux_term_vertical_ls.append(airdensityflux_term_vertical)
    turbulent_term_vertical_ls.append(turbulent_term_vertical)

## Calculate spatially averaged divergence values

In [ ]:
conservation_spatial_mean_df = pd.DataFrame({
    'advective_term_lateral' : [
        np.nanmean(item) for item in advective_term_lateral_ls
    ],
    'advective_term_vertical' : [
        np.nanmean(item) for item in advective_term_vertical_ls
    ],
    'advective_term_total' : [
        np.nanmean(item) for item in advective_term_total_ls
    ],
    'airdensityflux_term_vertical' : [
        np.nanmean(item) for item in airdensityflux_term_vertical_ls
    ],
    'turbulent_term_vertical' : [
        np.nanmean(item) for item in turbulent_term_vertical_ls
    ],
})
conservation_spatial_mean_df['time'] = timestamps
conservation_spatial_mean_df = conservation_spatial_mean_df.set_index('time')

In [ ]:
conservation_spatial_mean_df.to_csv("hi.csv")

In [ ]:
conservation_spatial_mean_df = pd.read_csv("hi.csv")
conservation_spatial_mean_df.time = pd.to_datetime(conservation_spatial_mean_df.time)

In [ ]:
line = alt.Chart().mark_rule().encode(y=alt.datum(0))

def conservation_spatial_mean_composite_plot(src, columns, title, times_filter = None, normalize = None):
    if times_filter is not None:
        src = src[src.time.isin(times_filter)]
    else:
        src = src
    if normalize is not None:
        src[columns] = src[columns] * normalize
    return line + alt.Chart(
        src.reset_index()
    ).transform_fold(
        columns
    ).mark_line().encode(
        alt.X('hoursminutes(time):T').title('time'),
        alt.Y('median(value):Q'),
        alt.Color('key:N')
    ).properties(height = 200, width = 200, title=title)

In [ ]:
(
    conservation_spatial_mean_composite_plot(
        conservation_spatial_mean_df,
        [
            'advective_term_lateral', 'advective_term_vertical', 'advective_term_total', 
            'airdensityflux_term_vertical', 'turbulent_term_vertical'
        ],
        title='All data', normalize = 7
    ) | conservation_spatial_mean_composite_plot(
        conservation_spatial_mean_df,
        [
            'advective_term_lateral', 'advective_term_vertical', 'advective_term_total', 
            'airdensityflux_term_vertical', 'turbulent_term_vertical'
        ],
        title='No BS, not snowing', normalize = 7,
        times_filter = set(nobs_times).intersection(set(is_not_snowing_dates))
    ) | conservation_spatial_mean_composite_plot(
        conservation_spatial_mean_df,
        [
            'advective_term_lateral', 'advective_term_vertical', 'advective_term_total', 
            'airdensityflux_term_vertical', 'turbulent_term_vertical'
        ],
        title='Not snowing', normalize = 7,
        times_filter = set(is_not_snowing_dates)
    ) | conservation_spatial_mean_composite_plot(
        conservation_spatial_mean_df,
        [
            'advective_term_lateral', 'advective_term_vertical', 'advective_term_total', 
            'airdensityflux_term_vertical', 'turbulent_term_vertical'
        ],
        title='BS', normalize = 7,
        times_filter = set(bs_times)
    )
).resolve_scale(y='shared', x='shared', color='shared').display(renderer='svg')

In [ ]:
alt.Chart(
    tidy_df[tidy_df.variable.isin(['w_h2o__3m_c', 'w_h2o__5m_c', 'w_h2o__10m_c'])]
).mark_line().encode(
    alt.X('hoursminutes(time):T'),
    alt.Y('median(value):Q'),
    alt.Color('height:O')
).properties(width = 400, height = 200)

In [ ]:
alt.Chart(
    tidy_df.query(
        "measurement == ''"
    ).query(
        "tower == 'c'"
    ).query("height <= 10").set_index('time').loc['20221221': '20221223'].reset_index()
).mark_line().encode(
    alt.X('time:T'),
    alt.Y('value:Q'),
    alt.Color('height:O')
).properties(width = 400, height = 200)

In [ ]:

src1 = conservation_spatial_mean_df.set_index('time').loc['20221221': '20221223'].reset_index().assign(casestudy = 1)
src2 = conservation_spatial_mean_df.set_index('time').loc['20221212': '20221214'].reset_index().assign(casestudy = 2)
alt.Chart(pd.concat([src1, src2])).transform_fold(
    src.columns.drop('time').tolist()
).mark_line().encode(
    alt.X('time:T'),
    alt.Y('value:Q'),
    alt.Color('key:N').scale(
        domain = ['advective_term_total', 'advective_term_lateral', 'advective_term_vertical',  'turbulent_term_vertical', 'airdensityflux_term_vertical'],
        range = ['black', 'darkgrey', 'light', 'red', 'blue']
    ),
    alt.Facet('casestudy:N').title(None)
).properties(width = 400, height = 200).resolve_scale(x='independent')

In [ ]:
alt.Chart(
    tidy_df.query(
        "measurement == 'wind direction'"
    ).query(
        "tower == 'c'"
    ).query("height <= 10").set_index('time').loc['20221221': '20221223'].reset_index()
).mark_line().encode(
    alt.X('time:T'),
    alt.Y('value:Q'),
    alt.Color('height:O')
).properties(width = 400, height = 200)

In [ ]:
(
    conservation_spatial_mean_composite_plot([
            'div_wind_field',
            'div_wind_field_lateral',
            'div_wind_field_vertical',
        ], 'Divergence of wind field (1/s)',
        set(nobs_times).intersection(set(is_not_snowing_dates))
    ) | (
        conservation_spatial_mean_composite_plot([
            'div_advectedflux_field',
            'div_advectedflux_field_lateral',
            'div_advectedflux_field_vertical',
        ], 'Divergence of advective flux (g/m^3)',
        set(nobs_times).intersection(set(is_not_snowing_dates))
        ) | conservation_spatial_mean_composite_plot([
                'div_turbulentflux_field',
                'div_turbulentflux_field_lateral',
                'div_turbulentflux_field_vertical',
            ], 'Divergence of turbulent flux (g/m^3)',
        set(nobs_times).intersection(set(is_not_snowing_dates))
        )
    ).resolve_scale(color='independent', x='shared', y='shared')
).resolve_scale(
    color='independent', x='shared', y='independent',
).configure_legend(orient='top', columns=1).properties(
    title='All data'
).display(renderer='svg')

In [ ]:
src = tidy_df[
    tidy_df.time.isin(
        set(nobs_times).intersection(set(is_not_snowing_dates))
    )
]
src = src[src.variable.isin(['w_h2o__3m_c', 'w_h2o__5m_c', 'w_h2o__10m_c'])]
ec_flux_chart = alt.Chart(
    src
).mark_line().encode(
    alt.X('hoursminutes(time):T'),
    alt.Y('median(value):Q'),
    alt.Color('height:O')
).properties(height = 200, width = 200, title='Vertical turbulent flux (g/m^2/s)')

In [ ]:
alt.Chart(
    tidy_df.query("measurement == 'air density'").query("height > 0")
).mark_line().encode(
    alt.X('hoursminutes(time):T'),
    alt.Y('median(value):Q').scale(zero=False),
    alt.Color('height:O')
) | alt.Chart(
    tidy_df.query("measurement == 'temperature'").query("height > 0")
).mark_line().encode(
    alt.X('hoursminutes(time):T'),
    alt.Y('median(value):Q').scale(zero=False),
    alt.Color('height:O')
)